In [79]:
!pip install google-api-python-client

In [80]:
!pip install --upgrade pip

In [81]:
import sys
sys.path

['/Users/gonakazawa/Documents/go/dev_environments/04_YouTube_analysis',
 '/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python310.zip',
 '/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10',
 '/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload',
 '',
 '/opt/homebrew/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages',
 '/opt/homebrew/opt/six/lib/python3.10/site-packages',
 '/opt/homebrew/lib/python3.10/site-packages',
 '/Users/gonakazawa/.pyenv/versions/3.10.6/lib/python3.10/site-packages']

In [82]:
sys.path.append("/Users/gonakazawa/.pyenv/versions/3.10.6/lib/python3.10/site-packages")

In [83]:
import sys
sys.path

['/Users/gonakazawa/Documents/go/dev_environments/04_YouTube_analysis',
 '/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python310.zip',
 '/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10',
 '/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload',
 '',
 '/opt/homebrew/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages',
 '/opt/homebrew/opt/six/lib/python3.10/site-packages',
 '/opt/homebrew/lib/python3.10/site-packages',
 '/Users/gonakazawa/.pyenv/versions/3.10.6/lib/python3.10/site-packages',
 '/Users/gonakazawa/.pyenv/versions/3.10.6/lib/python3.10/site-packages']

In [84]:
#!/usr/bin/python

from apiclient.discovery import build
#from apiclient.errors import HttpError
#from oauth2client.tools import argparser




In [85]:
import json
with open('secret.json') as f:
    secret = json.load(f)

In [86]:
DEVELOPER_KEY = secret['KEY']
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

# def youtube_search(options):
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
                developerKey=DEVELOPER_KEY)

q = 'python 自動化'
max_results = 50

response = youtube.search().list(
    q=q,
    part="id,snippet",
    order='viewCount',
    type='video',
    maxResults=max_results
).execute()


In [87]:
import pandas as pd

In [88]:
items_id = []
items = response['items']
for item in items:
    item_id = {}
    item_id['video_id'] = item['id']['videoId']
    item_id['channel_id'] = item['snippet']['channelId']
    items_id.append(item_id)

df_video = pd.DataFrame(items_id)

In [89]:
df_video[:3]

,video_id,channel_id
0,bsq2YY-XAEo,UC0xRMqPOyRNPTaL6BxhbCnQ
1,s8XjEuplx_U,UC8butISFwT-Wl7EV0hUK0BQ
2,PXMJ6FS7llk,UC8butISFwT-Wl7EV0hUK0BQ


In [90]:
def video_search(youtube, q = 'python 自動化',max_results = 50):

        response = youtube.search().list(
            q=q,
            part="id,snippet",
            order='viewCount',
            type='video',
            maxResults=max_results
        ).execute()

        items_id = []
        items = response['items']
        for item in items:
            item_id = {}
            item_id['video_id'] = item['id']['videoId']
            item_id['channel_id'] = item['snippet']['channelId']
            items_id.append(item_id)

        df_video = pd.DataFrame(items_id)
        return df_video

In [91]:
DEVELOPER_KEY = secret['KEY']
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"


youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
                developerKey=DEVELOPER_KEY)

q = 'python 自動化'
max_results = 50


In [92]:
video_search(youtube,q,max_results)[:3]

,video_id,channel_id
0,bsq2YY-XAEo,UC0xRMqPOyRNPTaL6BxhbCnQ
1,s8XjEuplx_U,UC8butISFwT-Wl7EV0hUK0BQ
2,PXMJ6FS7llk,UC8butISFwT-Wl7EV0hUK0BQ


In [93]:
channel_ids = df_video['channel_id'].unique().tolist()

subscriber_list = youtube.channels().list(
    id=','.join(channel_ids),
    part='statistics',
    fields='items(id,statistics(subscriberCount))'
).execute()

In [94]:
subscriber_list['items'][:4]

[{'id': 'UC6OrQk8WsnCOR1OezlUU9zQ', 'statistics': {'subscriberCount': '7650'}},
 {'id': 'UCk-AjDBOodw8P40f0y6EYeA',
  'statistics': {'subscriberCount': '93600'}},
 {'id': 'UCXjTiSGclQLVVU83GVrRM4w',
  'statistics': {'subscriberCount': '1550000'}},
 {'id': 'UCMMjv61LfBy5J3AT8Ua0NGQ',
  'statistics': {'subscriberCount': '145000'}}]

In [95]:
subscribers = []
for item in subscriber_list['items']:
    subscriber = {}
    subscriber['channel_id'] = item['id']
    subscriber['subscriber_count'] = int(item['statistics']['subscriberCount'])
    subscribers.append(subscriber)

df_subscribers = pd.DataFrame(subscribers)

In [96]:
df = pd.merge(left=df_video, right=df_subscribers, on='channel_id')
df.head(3)

,video_id,channel_id,subscriber_count
0,bsq2YY-XAEo,UC0xRMqPOyRNPTaL6BxhbCnQ,129000
1,KmqEyT7HznQ,UC0xRMqPOyRNPTaL6BxhbCnQ,129000
2,VRFfAeW30qE,UC0xRMqPOyRNPTaL6BxhbCnQ,129000


In [97]:
df_extracted = df[df['subscriber_count'] < 10000]
df_extracted[:3]

,video_id,channel_id,subscriber_count
30,YwIX8w9i2DM,UCbg0ptFI3yZ4Pe5vmUiEzzA,7550
39,YbGAUEjTKg4,UC6OrQk8WsnCOR1OezlUU9zQ,7650
40,uaIUOQ47eP8,UCEq0oQsS-voRnSWbcviIDGA,8200


In [98]:
video_ids = df_extracted['video_id'].tolist()

In [99]:
videos_list = youtube.videos().list(
    id=','.join(video_ids),
    part='snippet,statistics',
    fields='items(id,snippet(title),statistics(viewCount))'
).execute()

In [100]:
videos_info = []
items = videos_list['items']
for item in items:
    video_info = {}
    video_info['video_id'] = item['id']
    video_info['title'] = item['snippet']['title']
    video_info['view_count'] = item['statistics']['viewCount']
    videos_info.append(video_info)

df_videos_info = pd.DataFrame(videos_info)

In [101]:
df_videos_info[:3]

,video_id,title,view_count
0,YwIX8w9i2DM,Automate Excel with Python Tutorial (2020),274430
1,YbGAUEjTKg4,Web Automation using Python To Fill out A Form,115477
2,uaIUOQ47eP8,Google IT Automation with Python Professional ...,86267


In [102]:
results = pd.merge(left=df_extracted, right=df_videos_info, on='video_id')
results[:3]

,video_id,channel_id,subscriber_count,title,view_count
0,YwIX8w9i2DM,UCbg0ptFI3yZ4Pe5vmUiEzzA,7550,Automate Excel with Python Tutorial (2020),274430
1,YbGAUEjTKg4,UC6OrQk8WsnCOR1OezlUU9zQ,7650,Web Automation using Python To Fill out A Form,115477
2,uaIUOQ47eP8,UCEq0oQsS-voRnSWbcviIDGA,8200,Google IT Automation with Python Professional ...,86267


In [103]:
results = results.loc[:,['video_id', 'title', 'view_count', 'subscriber_count', 'channel_id']]

In [104]:
results[:3]

,video_id,title,view_count,subscriber_count,channel_id
0,YwIX8w9i2DM,Automate Excel with Python Tutorial (2020),274430,7550,UCbg0ptFI3yZ4Pe5vmUiEzzA
1,YbGAUEjTKg4,Web Automation using Python To Fill out A Form,115477,7650,UC6OrQk8WsnCOR1OezlUU9zQ
2,uaIUOQ47eP8,Google IT Automation with Python Professional ...,86267,8200,UCEq0oQsS-voRnSWbcviIDGA
